# Manipulando dados no excel

Usaremos 2 bibliotecas para manipulação de dados:
 * [openpyxl](https://openpyxl.readthedocs.io/en/stable/)
 * [pandas](https://pandas.pydata.org/)

### Usaremos agora a **pandas**

In [162]:
import pandas as pd
import os

#### Indicando o arquivo em excel
_Obs:_ É possível usar tanto extensões od tipo ".xlsx" e ".csv". Trocar o nome do arquivo lido para fazer a filtragem.

In [163]:
#workSpace = pd.read_excel("data/consumo_sul_do_brasil.xlsx", sheet_name="Worksheet")
workSpace = pd.read_excel("data/consumo_sul_do_brasil.xlsx", sheet_name="Worksheet")

#### Podemos ver o tamanho da planilha usando o método "shape()"

In [164]:
print(workSpace.shape)

(8766, 5)


Também podemos retirar as colunas que não nos interessam, como:
 * Dia de Data 
 * Período Exibido
 * Subsistema

In [165]:
# Removendo as colunas que não serão utilizadas em nossa análise
#! Sul
#workSpace = workSpace.drop(columns=["Dia de Data Escala de Tempo 1 CE Simp 4", "Período Exibido CE Simp 4", "Subsistema"])
#print(workSpace.shape)

#! Norte
workSpace = workSpace.drop(columns=["Dia de Data Escala de Tempo 1 CE Simp 4", "Período Exibido CE Simp 4", "Subsistema"])
print(workSpace.shape)

(8766, 2)


Agora possuímos apenas dados referentes ao município de Florianópolis. Porém a distribuição destes dados no DataFrame não está de forma ideal para analisá-los. Utilizaremos a função **melt()** para despivotar a tabela utilizando a coluna classe como referência, assim alterando o formato de **wide** para **long**

In [166]:
# Renomeando as colunas restantes para refletir o que estamos analisando
workSpace.rename(columns = {"Data Escala de Tempo 1 CE Simp 4":"Datetime", "Selecione Tipo de CE Simp 4":"MWh"}, inplace = True)
print(workSpace)

                 Datetime         MWh
0     01/01/1999 00:00:00   109,74888
1     02/01/1999 00:00:00   118,83792
2     03/01/1999 00:00:00   117,29904
3     04/01/1999 00:00:00   152,29632
4     05/01/1999 00:00:00   153,91848
...                   ...         ...
8761  27/12/2022 00:00:00  279,059467
8762  28/12/2022 00:00:00  264,907205
8763  29/12/2022 00:00:00  255,616802
8764  30/12/2022 00:00:00  257,337626
8765  31/12/2022 00:00:00  245,716996

[8766 rows x 2 columns]


### trocando "/" por "-" e "00:00:00" por "01:00:00"

In [167]:
workSpace['Datetime'] = workSpace['Datetime'].str.replace('/', '-')
workSpace['Datetime'] = workSpace['Datetime'].str.replace('00:00:00', '01:00:00')
print(workSpace)

                 Datetime         MWh
0     01-01-1999 01:00:00   109,74888
1     02-01-1999 01:00:00   118,83792
2     03-01-1999 01:00:00   117,29904
3     04-01-1999 01:00:00   152,29632
4     05-01-1999 01:00:00   153,91848
...                   ...         ...
8761  27-12-2022 01:00:00  279,059467
8762  28-12-2022 01:00:00  264,907205
8763  29-12-2022 01:00:00  255,616802
8764  30-12-2022 01:00:00  257,337626
8765  31-12-2022 01:00:00  245,716996

[8766 rows x 2 columns]


## Convertendo a data de "dd-mm-yyyy" para "yyyy-mm-dd"

In [168]:
# Realizar a alteração na coluna desejada
workSpace['Datetime'] = pd.to_datetime(workSpace['Datetime'], format='%d-%m-%Y %H:%M:%S')

workSpace['Datetime'] = workSpace['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(workSpace)

                 Datetime         MWh
0     1999-01-01 01:00:00   109,74888
1     1999-01-02 01:00:00   118,83792
2     1999-01-03 01:00:00   117,29904
3     1999-01-04 01:00:00   152,29632
4     1999-01-05 01:00:00   153,91848
...                   ...         ...
8761  2022-12-27 01:00:00  279,059467
8762  2022-12-28 01:00:00  264,907205
8763  2022-12-29 01:00:00  255,616802
8764  2022-12-30 01:00:00  257,337626
8765  2022-12-31 01:00:00  245,716996

[8766 rows x 2 columns]


#### Convertendo para o formato ".csv"

1) Concatenando as colunas "Datetime" e "MWh"

In [169]:
# workSpace_concat = pd.concat([workSpace['Datetime'], workSpace['MWh']], axis=1)
# print(workSpace_concat)

## Trocando "," por "."

In [170]:
workSpace['MWh'] = workSpace['MWh'].str.replace(',', '.')
print(workSpace)

                 Datetime         MWh
0     1999-01-01 01:00:00   109.74888
1     1999-01-02 01:00:00   118.83792
2     1999-01-03 01:00:00   117.29904
3     1999-01-04 01:00:00   152.29632
4     1999-01-05 01:00:00   153.91848
...                   ...         ...
8761  2022-12-27 01:00:00  279.059467
8762  2022-12-28 01:00:00  264.907205
8763  2022-12-29 01:00:00  255.616802
8764  2022-12-30 01:00:00  257.337626
8765  2022-12-31 01:00:00  245.716996

[8766 rows x 2 columns]


## Multiplicando os dados de MWh por 10000

In [171]:
# workSpace_concat['MWh'] = workSpace_concat['MWh'].astype(float) * 10000.0
# print(workSpace_concat)
workSpace['MWh'] = workSpace['MWh'].astype(float).apply(lambda x: round(x, 1))
print(workSpace)

                 Datetime    MWh
0     1999-01-01 01:00:00  109.7
1     1999-01-02 01:00:00  118.8
2     1999-01-03 01:00:00  117.3
3     1999-01-04 01:00:00  152.3
4     1999-01-05 01:00:00  153.9
...                   ...    ...
8761  2022-12-27 01:00:00  279.1
8762  2022-12-28 01:00:00  264.9
8763  2022-12-29 01:00:00  255.6
8764  2022-12-30 01:00:00  257.3
8765  2022-12-31 01:00:00  245.7

[8766 rows x 2 columns]


## Pegando a parte inteira

## Transformando em float dnv

5) Convertendo no formato ".csv" e salvando os modelos filtrados.

_Obs_: Trocar o nome do arquivo para o mesmo nome do arquivo lido para fazer o salvamento.

In [172]:
# Salvando as alterações concatenadas
workSpace.to_csv("data/After Filters/gasto_energia_sul.csv", index=False)
print(workSpace)

                 Datetime    MWh
0     1999-01-01 01:00:00  109.7
1     1999-01-02 01:00:00  118.8
2     1999-01-03 01:00:00  117.3
3     1999-01-04 01:00:00  152.3
4     1999-01-05 01:00:00  153.9
...                   ...    ...
8761  2022-12-27 01:00:00  279.1
8762  2022-12-28 01:00:00  264.9
8763  2022-12-29 01:00:00  255.6
8764  2022-12-30 01:00:00  257.3
8765  2022-12-31 01:00:00  245.7

[8766 rows x 2 columns]
